# TAREFA DATASET KAGGLE


Para esta fase, o objetivo é alcançar o melhor resultado possível com o dataset da competição fornecido pelos docentes na previsão do nível de incidentes rodoviários, numa determinada hora, na cidade de Guimarães.

### **1.** Importar as bibliotecas essenciais do Python para a elaboração desta tarefa

In [ ]:
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

pd.set_option('display.max_columns', None)

### **2.** Carregar o dataset para um dataframe da biblioteca Pandas

In [ ]:
df = pd.read_csv('docs/training_data.csv', encoding="utf-8", skipinitialspace=True)
df_teste = pd.read_csv('docs/test_data.csv', encoding="utf-8", skipinitialspace=True)

### **3.** Obtenção de informação acerca do dataset: tipos de dados das features, conteúdo do dataset e estatística

* **tipos de dados das features**

In [ ]:
df.info()

* **conteúdo do dataset**

In [ ]:
df.head()

* **estatística**

In [ ]:
df.describe()

Distribuição da feature target "incidents"

In [ ]:
df["incidents"].hist()

Análise dos valores únicos no dataset

In [ ]:
for c in df:
    print(f"{c}: {df[c].unique()}")
    print(f"Quantidade: {df[c].nunique()}")
    print("---------------------------------------")

TODO: tabelinha gira da análise da tabela acima

### **4.** Preparação dos dados

In [ ]:
df.head()

In [ ]:
df.columns

**Remoção da feature "avg_precipitation"**

In [ ]:
#Possui informação incompleta
df = df.drop('avg_precipitation', axis=1)
df_teste = df_teste.drop('avg_precipitation', axis=1)

**Transformação da coluna *record_date* nas colunas ano, mês, dia, hora e minuto**

In [ ]:
df['record_date'] = pd.to_datetime(df['record_date'],format='%Y-%m-%d %H:%M', errors='coerce')
assert df['record_date'].isnull().sum() == 0,'missing record date'

df['record_date_year'] = df['record_date'].dt.year
df['record_date_month'] = df['record_date'].dt.month
df['record_date_day'] = df['record_date'].dt.day
df['record_date_hour'] = df['record_date'].dt.hour
df['record_date_minute'] = df['record_date'].dt.minute
df = df.drop('record_date', axis=1)

df.head()


df_teste['record_date'] = pd.to_datetime(df_teste['record_date'],format='%Y-%m-%d %H:%M', errors='coerce')
assert df_teste['record_date'].isnull().sum() == 0,'missing record date'

df_teste['record_date_year'] = df_teste['record_date'].dt.year
df_teste['record_date_month'] = df_teste['record_date'].dt.month
df_teste['record_date_day'] = df_teste['record_date'].dt.day
df_teste['record_date_hour'] = df_teste['record_date'].dt.hour
df_teste['record_date_minute'] = df_teste['record_date'].dt.minute
df_teste = df_teste.drop('record_date', axis=1)

**Remoção das features "city_name", "record_date_year" e "record_date_minute"**

In [ ]:
#como apenas existe um valor para "city_name", esta coluna pode ser removida
df = df.drop('city_name', axis=1)
df_teste = df_teste.drop('city_name', axis=1)

#o mesmo se aplica para o ano e minuto
df = df.drop('record_date_year', axis=1)
df = df.drop('record_date_minute', axis=1)

df_teste = df_teste.drop('record_date_year', axis=1)
df_teste = df_teste.drop('record_date_minute', axis=1)

**Transformação da feature "magnitude_of_delay" em valores numéricos**

In [ ]:
df['magnitude_of_delay'].unique()

In [ ]:
replace_map = {'magnitude_of_delay': {'UNDEFINED':0 ,'MODERATE': 1, 'MAJOR': 2}}
df.replace(replace_map,inplace=True)
df_teste.replace(replace_map,inplace=True)
df.head()

**Tratamento da feature "affected_roads"**

* Tratamento dos missing values

In [ ]:
df['affected_roads'].fillna(',',inplace=True) #tratar de missing values
df_teste['affected_roads'].fillna(',',inplace=True) #tratar de missing values
df.info()

* Separação das ruas e remoção de ruas repetidas

In [ ]:
affected_roads = {}

#Separação das ruas e remoção de ruas repetidas
for row in df['affected_roads']:
    roads = list(dict.fromkeys(row.split(',')))
    for road in roads:
        affected_roads[road] = []

for row in df['affected_roads']:
    roads = list(dict.fromkeys(row.split(',')))
    for road in affected_roads:
        if road in roads:
            affected_roads[road].append(1)
        else:
            affected_roads[road].append(0)

        
affected_roads = pd.DataFrame.from_dict(affected_roads)
df = df.drop('affected_roads',axis=1)

for column in affected_roads:
    df[column] = affected_roads[column]
    
#######################################################################
affected_roads = {}

for row in df_teste['affected_roads']:
    roads = list(dict.fromkeys(row.split(',')))
    for road in roads:
        affected_roads[road] = []

for row in df_teste['affected_roads']:
    roads = list(dict.fromkeys(row.split(',')))
    for road in affected_roads:
        if road in roads:
            affected_roads[road].append(1)
        else:
            affected_roads[road].append(0)

        
affected_roads = pd.DataFrame.from_dict(affected_roads)
df_teste = df_teste.drop('affected_roads',axis=1)

for column in affected_roads:
    df_teste[column] = affected_roads[column]
    
df

**Transformação da feature "luminosity" em valores numéricos**

In [ ]:
df['luminosity'].unique()

In [ ]:
replace_map = {'luminosity': {'LOW_LIGHT':0 ,'LIGHT': 1, 'DARK': 2}}
df.replace(replace_map,inplace=True)
df_teste.replace(replace_map,inplace=True)
df.head()

**Tranformação da feature "avg_rain" em valores numéricos**

In [ ]:
df['avg_rain'].unique()

In [ ]:
replace_map = {'avg_rain': {'Sem Chuva':0 ,'chuva fraca': 1, 'chuva moderada': 2,'chuva forte' : 3}}
df.replace(replace_map,inplace=True)
df_teste.replace(replace_map,inplace=True)
df.head()

**Transformação da feature "incidents" em valores numéricos**

In [ ]:
replace_map = {'incidents': {'None':0 ,'Low': 1, 'Medium': 2, 'High': 3, 'Very_High': 4}}
df.replace(replace_map,inplace=True)
df_teste.replace(replace_map,inplace=True)
df.head()

### 5. Aplicação de modelos de machine learning

#### 5.1. Decision Tree Classifier

Imports necessários para a implementação deste modelo

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report, plot_confusion_matrix
from sklearn import tree

In [ ]:
x = df.drop("incidents", axis = 1)

In [ ]:
y = df.incidents

In [ ]:
x_train, x_test, y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=2022)

Treino do modelo

In [ ]:
clf = DecisionTreeClassifier(random_state=2022)
clf = clf.fit(x_train,y_train)

Visualização da árvore de decisão

In [ ]:
tree.export_graphviz(clf,
                     out_file="tree_dsCompeticao.dot",
                     filled = True)

from graphviz import Source
Source.from_file("tree_dsCompeticao.dot")

Geração de previsões

In [ ]:
predictions = clf.predict(x_test)
pd.DataFrame(predictions)

In [ ]:
y_test

Avaliação do modelo

In [ ]:
plot_confusion_matrix(clf, x_test, y_test) 

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
mean_absolute_error(y_test,predictions)

**Geração do ficheiro de submissão para o Decision Tree**

In [ ]:
#Função genérica que irá ser reutilizada para os proximos modelos
def download_submission_file(model, df_teste, filename):
    predictions_dfT = model.predict(df_teste)
    pd.DataFrame(predictions_dfT)
    
    replace_map = {'Incidents': {"0":'None' ,"1":'Low', "2":'Medium', "3":'High', "4":'Very_High'}}
    
    predictions_dfT = pd.DataFrame(predictions_dfT, columns = ["Incidents"])
    predictions_dfT["Incidents"] = predictions_dfT["Incidents"].astype(str)
    predictions_dfT.replace(replace_map,inplace=True)
    predictions_dfT.index+=1
    predictions_dfT.head()
    
    from pathlib import Path
    filepath = Path(filename)
    predictions_dfT.to_csv(filepath, index = True)

In [ ]:
download_submission_file(clf, df_teste, "submission_files/decisionTree.csv")

#### 5.2 Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [ ]:
x = df.drop("incidents", axis = 1)
x

In [521]:
y = df.incidents
y

0       0
1       0
2       1
3       4
4       3
       ..
4995    3
4996    0
4997    0
4998    0
4999    0
Name: incidents, Length: 5000, dtype: int64

In [522]:
x_train, x_test, y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=2022)

In [530]:
lr = LogisticRegression(random_state=2022, solver="liblinear") #mudar solver?

In [531]:
lr.fit(x_train,y_train)

LogisticRegression(random_state=2022, solver='liblinear')

In [532]:
predictions_lr = lr.predict(x_test)
predictions_lr

array([0, 1, 0, ..., 3, 3, 3], dtype=int64)

In [533]:
y_test

3419    1
178     4
3721    0
3087    0
3685    1
       ..
1266    2
4556    4
1371    2
4911    3
3642    3
Name: incidents, Length: 1250, dtype: int64

In [534]:
print(classification_report(y_test, predictions_lr))

              precision    recall  f1-score   support

           0       0.82      0.98      0.89       521
           1       0.58      0.41      0.48       169
           2       0.62      0.03      0.07       144
           3       0.60      0.78      0.68       262
           4       0.79      0.78      0.79       154

    accuracy                           0.73      1250
   macro avg       0.68      0.60      0.58      1250
weighted avg       0.71      0.73      0.68      1250



In [535]:
download_submission_file(lr, df_teste, "submission_files/logisticRegression.csv")

#### 5.3 Support Vector Machine

#### 5.4 Redes neuronais

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler

RANDOM_SEED = 2022

print("TensorFlow version:", tf.__version__)

In [ ]:
x = df.drop('incidents',axis=1)
y = df[['incidents']]

In [ ]:
df.info

In [ ]:
df_teste